In [4]:
from langchain_upstage import UpstageDocumentParseLoader, UpstageGroundednessCheck, ChatUpstage, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.utils.math import cosine_similarity

from tokenizers import Tokenizer
from IPython.display import display, HTML

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [6]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [8]:
%load_ext dotenv
%dotenv

In [9]:
doc_loader_text = UpstageDocumentParseLoader("./laws-of-the-game-2024-25-korean-en.pdf", output_format='text', coordinates=False)
docs = doc_loader_text.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

print("Splits:", len(splits)) 

Splits: 113


In [11]:
print(splits[0].page_content)

경기규칙Laws of the Game24/25대한축구협회 공식 번역판 The Official Translation of Korea Football Association국제축구평의회
스위스 취리히 뮌스터 9가 8001
전화 : +41 (0)44 245 1886 팩스 : +41 (0)44 245 1887
www.theifab.com이 책자는 국제축구평의회(IFAB)의 허락 없이 전체 또는 일부를
복제하거나 번역할 수 없습니다.본 규정은 2024년 7월 1일부터 효력이 발생합니다.경기규칙Laws of the Game2024/25우리말을 쓰는
전 세계의
축구 가족들에게우리말은 늘 여러 이름으로 불립니다. 그것은 조선말( 北三省朝鲜语, 在日朝鮮語)이기도,
고려말(Корё мар)이기도 하며 한국말이라고 불리기도 합니다. 우리 민족에게 20세기가 헤
어짐과 만남의 반복으로 얼룩진 한 세기였듯, 우리말의 이름 또한 여러 다른 이름으로 갈라
지고 말았습니다. 우리말의 이름이 여러 가지로 흩어졌음에도, 그러나 축구는 언제나 하나
였으며, 그 변하지 않은 축구는 늘 우리 겨레와 함께했습니다. 그리고 그 “축구의 법(Laws
of The Game)”도 늘 하나였습니다.1928년 5월 22일 서울에서 조직된 ‘조선심판협회’는 대한축구협회의 어머니가 되었습니다.
대한축구협회의 탄생을 보듬은 모태가 ‘심판(審判, match officials)’의 모임이었다는 역사
적 진실에 비추어 보건대, “경기규칙서(Laws of The Game)”를 우리말로 옮겨오는 일이 우
리들 심판에게 주어진 으뜸가는 임무이자, 그 태생부터 주어진 숙명이었음은 너무나 분명
합니다.지난 몇 년에 걸쳐, 경기규칙서를 우리말로 옮기는 작업에는 저 먼 남쪽의 제주 섬사람부터,
광양, 곡성, 대구, 대전, 전주, 수원에 이르는 각처에서 나고 자란 사람들이 참여했습니다. 또
한 그 나이는 서른을 넘긴 사람부터 환갑에 이르는 사람까지 걸쳐있습니다. 이렇게 우리말


In [12]:
u_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [13]:
vectorstore = Chroma.from_documents(documents=splits, embedding=u_embeddings)

In [14]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [15]:
tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")

In [16]:
rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            너는 축구 용어에 대한 질문에 답하는 AI 챗봇이야.
            제공된 문서를 참고하고, 질문 히스토리에 기반해서 답변해줘.
            답변을 모르면 그냥 모른다고 답해줘.
            ---
            CONTEXT:
            {context}
            """
        ), 
        MessagesPlaceholder(variable_name='history'), 
        ("human", "{input}"),
    ]
)

In [17]:
llm = ChatUpstage()

In [18]:
chain = rag_with_history_prompt | llm | StrOutputParser()

In [19]:
def get_relevant_context(question, retriever):
    # retriever를 사용해 관련된 문서만 가져옴
    relevant_docs = retriever.get_relevant_documents(question)
    
    # 관련 문서들을 하나의 문자열로 합침
    return "\n".join(doc.page_content for doc in relevant_docs)


In [20]:
chat_history = []
question1 = "오프사이드 규칙에 대해 설명해줘"

relevant_context_1 = get_relevant_context(question1, retriever)
response1 = chain.invoke({"history": chat_history, "context": relevant_context_1, "input": question1})

print("RESPONSE1\n", response1) 

RESPONSE1
 오프사이드 규칙은 축구 경기에서 공격측이 수비측보다 유리한 위치에서 공을 받는 것을 방지하기 위한 규칙입니다. 이 규칙에 따르면, 공격측 선수가 상대팀 골대 근처에서 공을 받을 때, 상대팀 골대 근처에 있는 수비측 선수보다 더 가까운 위치에서 공을 받으면 오프사이드로 판단됩니다. 

오프사이드 규칙은 게임의 균형을 유지하고, 수비측이 공격측에 대한 압박을 유지할 수 있도록 도와줍니다. 이를 통해 게임의 공정성과 전략적인 요소를 더해줍니다.


In [21]:
chat_history.extend([HumanMessage(content=question1), AIMessage(content=response1)])
question2 = "그럼 오프사이드 위치에 있더라도 실제 오프사이드가 아닌 경우는 언제야?"

relevant_context_2 = get_relevant_context(question1, retriever)
response2 = chain.invoke({"history": chat_history, "context": relevant_context_2, "input": question2})

print("RESPONSE2\n", response2)

RESPONSE2
 오프사이드 위치에 있더라도 실제 오프사이드가 아닌 경우는 다음과 같습니다:

1. 공을 직접 플레이하지 않은 경우: 오프사이드 위치에 있더라도 공을 직접 플레이하지 않고 상대 팀에게 이점을 주지 않았다면 오프사이드가 아닙니다.
2. 공을 마지막으로 플레이한 팀 동료의 플레이에 방해가 되지 않은 경우: 팀 동료가 공을 마지막으로 플레이한 시점에 오프사이드 위치에 있었지만, 그 플레이에 방해가 되지 않았다면 오프사이드가 아닙니다.
3. 골키퍼가 의도적으로 공을 던진 후 오프사이드 위치에 들어간 경우: 골키퍼가 공을 던진 후에는 상대 팀이 오프사이드 위치에 있을 수 있습니다.
4. 공을 마지막으로 플레이한 팀 동료의 플레이에 방해가 되지 않은 경우: 팀 동료가 공을 마지막으로 플레이한 시점에 오프사이드 위치에 있었지만, 그 플레이에 방해가 되지 않았다면 오프사이드가 아닙니다.

이러한 경우에는 오프사이드 위치에 있더라도 실제 오프사이드가 아니므로 경기 진행에 영향을 주지 않습니다.
